In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from pathlib import Path

From the tutorial [Landsat for Geosciences](https://medium.com/analytics-vidhya/python-for-geosciences-raster-bit-masks-explained-step-by-step-8620ed27141e)

In [ ]:
def load_landsat_image(img_folder, bands):
    image = {}
    path = Path(img_folder)
    for band in bands:
        file = next(path.glob(f'*{band}.tif'))
        print(f'Opening file {file}')
        ds = rasterio.open(file)
        image.update({band: ds.read(1)})
    return image

img = load_landsat_image('F:/Data/Spatial - Raster/Imagery/Afghanistan/LC08_L2SP_154038_20181229_20200829_02_T1_ST/', ['B2', 'B3', 'B4', 'QA_PIXEL'])

img['QA_PIXEL']

In [ ]:
mask_values = {
               'Dilated cloud over land': 21826,
               'Dilated cloud over water': 21890,
               'Mid conf Cloud': 22280,
               'High conf cloud shadow': 23888,
               'Water with cloud shadow': 23952,
               'Mid conf cloud with shadow': 24088,
               'Mid conf cloud with shadow over water': 24216,
               'High conf cloud with shadow': 24344,
               'High conf cloud with shadow over water': 24472,
               'High conf Cirrus': 54596,
               'Cirrus, high cloud': 55052,
               'Cirrus, mid conf cloud, shadow': 56856,
               'Cirrus, mid conf cloud, shadow, over water': 56984,
               'Cirrus, high conf cloud, shadow': 57240,
              }
final_mask = np.zerow_like(img['QA_PIXEL'])

for key, value in mask_values.items():
    mask = (img['QA_PIXEL'] == value)
    count = np.count_nonzero(mask)
    print(f'{key}: {count}')
    final_mask = final_mask | mask

plt.figure(figsize=(7,7))
plt.imshow(final_mask[4000:5000, 5400:6400])

Accessing bit masks in Python

In [ ]:
mask_bit = np.bitwise_and(img['QA_PIXEL'], compare_mask)

for value in np.unique(mask_bit):
    print(f'{value:05b}')

In [ ]:
final_mask_bit = mask_bit > 0

plt.figure(figsize=(7,7))
plt.imshow(final_mask_bit[4000:5000, 5400:6400])

Simplify Landsat 8 mask processor

In [ ]:
L8_flags = {
    'dilated_cloud': 1<<1,
    'cirrus': 1<<2,
    'cloud': 1<<3,
    'shadow': 1<<4,
    'snow': 1<<5,
    'clear': 1<<6,
    'water': 1<<7
}

def get_mask(mask, flaglist):
    final_mask = np.zeros_like(mask)
    for flag in flag_list:
        flag_mask = np.bitwise_and(mask, L8_flags[flag])
        final_mask = final_mask | flag_mask
    return final_mask > 0

clouds = get_mask(img['QA_PIXEL'], ['cirrus', 'cloud', 'dilated_cloud'])
shadows = get_mask(img['QA_PIXEL'], ['shadow'])

fig, ax = plt.subsplots(1, 2, figsize=(15,7))
ax[0].imshow(clouds[4000:5000, 5400:6400])
ax[1].imshow(shadows[4000:5000, 5400:6400])

Overlay with RGB Image

In [ ]:
red = np.where(mask==True, red*transparency+color_mask[0]*(1-transparency), red)
green = np.where(mask==True, green*transparency+color_mask[1]*(1-transparency), green)
blue = np.where(mask==True, blue*transparency+color_mask[2]*(1-transparency), blue)

In [ ]:
def plot_masked_rgb(red, green, blue, mask, color_mask=(1, 0, 0), transparency=0.5, brightness=2):
    red = red / red.max() * brightness
    green = green / green.max() * brightness
    blue = blue / blue.max() * brightness 

    red = np.where(mask==True, red*transparency+color_mask[0]*(1-transparency), red)
    green = np.where(mask==True, green*transparency+color_mask[1]*(1-transparency), green)
    blue = np.where(mask==True, blue*transparency+color_mask[2]*(1-transparency), blue)

    rgb = np.stack([red, green, blue], axis=2)
    return rgb
    
rgb = plot_masked_rgb(img['B4'], img['B3'], img['B2'], shadows, color_mask=(1, 0, 0), transparency=0.7, brightness=2)
rgb = plot_masked_rgb(rgb[..., 0], rgb[..., 1], rgb[..., 2], clouds, color_mask=(1, 1, 0), transparency=0.7, brightness=2)

plt.figure(figsize=(12,12))
plt.imshow(rgb[4000:5000, 5400:6400, 0:3])